In [1]:
import random
import pickle
import heapq

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import RegexpTokenizer

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

2024-04-22 17:14:16.004026: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 17:14:16.004184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 17:14:16.196398: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:

with open("/kaggle/input/next-word-prediction/1661-0.txt", "r") as file:
    text = file.read()

text_df = pd.DataFrame({"text": [text]})

text = list(text_df.text.values)
joined_text = " ".join(text)

with open("joined_text.txt", "w", encoding="utf-8") as f:
    f.write(joined_text)

In [3]:
partial_text = joined_text[:1000000]

In [4]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [5]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: index for index, token in enumerate(unique_tokens)}

In [6]:
n_words = 10
input_words = []
next_word = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_word.append(tokens[i + n_words])

In [7]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)  # for each sample, n input words and then a boolean for each possible next word
y = np.zeros((len(next_word), len(unique_tokens)), dtype=bool)  # for each sample a boolean for each possible next word

In [8]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_word[i]]] = 1

In [9]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X, y, batch_size=356, epochs=10, shuffle=True).history

Epoch 1/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 260s 837ms/step - accuracy: 0.0498 - loss: 6.7804
Epoch 2/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 250s 813ms/step - accuracy: 0.0816 - loss: 6.0667
Epoch 3/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 248s 808ms/step - accuracy: 0.1141 - loss: 5.6501
Epoch 4/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 256s 835ms/step - accuracy: 0.1338 - loss: 5.3856
Epoch 5/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 246s 802ms/step - accuracy: 0.1487 - loss: 5.1709
Epoch 6/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 262s 803ms/step - accuracy: 0.1581 - loss: 5.0101
Epoch 7/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 264s 810ms/step - accuracy: 0.1756 - loss: 4.8002
Epoch 8/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 261s 805ms/step - accuracy: 0.1903 - loss: 4.6386
Epoch 9/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 248s 806ms/step - accuracy: 0.2077 - loss: 4.4606
Epoch 10/10
307/307 ━━━━━━━━━━━━━━━━━━━━ 246s 802ms/step - accuracy: 0.2231 - loss: 4.2908


In [11]:
history = model.fit(X, y, batch_size=260, epochs=5, shuffle=True).history

Epoch 1/5
421/421 ━━━━━━━━━━━━━━━━━━━━ 257s 611ms/step - accuracy: 0.2333 - loss: 4.1971
Epoch 2/5
421/421 ━━━━━━━━━━━━━━━━━━━━ 259s 615ms/step - accuracy: 0.2506 - loss: 4.0901
Epoch 3/5
421/421 ━━━━━━━━━━━━━━━━━━━━ 267s 633ms/step - accuracy: 0.2708 - loss: 3.9372
Epoch 4/5
421/421 ━━━━━━━━━━━━━━━━━━━━ 314s 614ms/step - accuracy: 0.2913 - loss: 3.7854
Epoch 5/5
421/421 ━━━━━━━━━━━━━━━━━━━━ 261s 612ms/step - accuracy: 0.3117 - loss: 3.6329


In [12]:
model.save("text_gen_model2.h5")
with open("history2.p", "wb") as f:
    pickle.dump(history, f)

In [13]:
model = load_model("text_gen_model2.h5")
history = pickle.load(open("history2.p", "rb"))

In [14]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1
        
    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [15]:
possible = predict_next_word("I will have to look into this thing because I", 5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step


In [16]:
for idx in possible:
    print(unique_tokens[idx])

will
should
have
am
shall


In [17]:
def generate_text(input_text, n_words, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(n_words):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [18]:
generate_text("I will have to look into this thing because I", 100, 10)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━

'I will have to look into this thing because I must not hear my help he never have some small matter upon this time which is a very kind well if do it had got in a man s a game smack chair his hand more pale people indeed something most intimate details as i have no one and two hundred point upon present well if they are not yet it will i am the field from mine i must do what all turner is that in every sort of help mr jabez sherlock she sat and as the second first about it or where they have had to be'

In [19]:
generate_text("The president of the United States announced yesterday that he", 100, 10)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━

'The president of the United States announced yesterday that he volcanic cannon is at our subject and that he would go it is sir george s man s young he could take her time to change so it seemed of his long little mary that before she might take it as a very heavy business round the stone with red thin coat face long then it s a wild chair he does indeed with your most retiring statement he has already remarked my father is all in that he he had his white hands of mine s the point out there and there has any reason for me and in'

In [21]:
for idx in predict_next_word("The man will most likely not be there to help", 5):
    print(unique_tokens[idx])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
that
it
any
the
their
